In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r'C:\Users\DELL\OneDrive\Desktop\Music_Rec\archive\spotify_millsongdata.csv')

In [3]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.shape

(57650, 4)

In [5]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [6]:
df=df.sample(2000).drop('link', axis=1).reset_index(drop=True)

In [7]:
df.head(5)

,artist,song,text
0,Wishbone Ash,Like A Child,The sky is changing \r\nThe clouds are gather...
1,Jason Mraz,3 Things,There are three things I do when my life falls...
2,Red Hot Chili Peppers,The Greeting Song,Drivin' around I've got my baby and my top dow...
3,Gary Numan,Fold,"I fold your letter \r\nAll lies, all cold \r..."
4,Doors,The Wasp,I wanna tell you 'bout texas radio and the big...


In [8]:
df.shape

(2000, 3)

In [9]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [10]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [11]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [14]:
similarity[0]

array([1.        , 0.07549009, 0.06073948, ..., 0.03001546, 0.03929087,
       0.01472237])

In [20]:
df[df['song'] == 'The Wasp']

,artist,song,text
4,Doors,The Wasp,i wan na tell you 'bout texa radio and the big...


In [16]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:6]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [18]:
recommendation('The Wasp')

['Tell Me',
 'Thinking Bout Somethin',
 "It's Not Just Me",
 'I Need Your Love',
 'Bout Me']

In [19]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))

In [ ]:
print(pickle.__version__)